In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import os
Image.MAX_IMAGE_PIXELS = None
%matplotlib inline

INPUT_FILE = "..." # replace input raster path
OUTPUT_FILE = "..." # replace output raster path

In [ ]:
with rasterio.open(INPUT_FILE) as dataset:
    transform = dataset.transform
    print(f"Transform: {transform}")
    pixelSizeX = dataset.transform[0]
    pixelSizeY = -dataset.transform[4]  # Note the negative sign for ysize
    print(f"Pixel Size X: {pixelSizeX}, Pixel Size Y: {pixelSizeY}")
    image_numpy = np.array(dataset.read(1))
    print(f"CRS: {dataset.crs}")
    print(dataset.meta['nodata'])
    print(dataset.count)
    plt.imshow(dataset.read(1), cmap='terrain')
    plt.show()

In [ ]:
from rasterio.enums import Resampling
# Determine the new resolution
new_resolution = None # replace with new resolution (e.g, 2.0)
old_resolution = None # replace with old resolution (e.g, 4.0)

with rasterio.open(INPUT_FILE) as dataset:

    # Nodata value
    nodata_value = dataset.meta['nodata']
    print(dataset.meta['dtype'])
    print(dataset.transform)
    print(dataset.count)

    # Calculate the upscale factor
    upscale_factor = old_resolution / new_resolution
    # Read the dataset's band into memory (consider reading in chunks if it's very large)
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear  # You can choose other resampling methods as well
    ).astype(np.float32)

    # Update the dataset's transform (affine matrix)
    new_transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

    with rasterio.Env():

        # Write an array as a raster band to a new 8-bit file. For
        # the new file's profile, we start with the profile of the source
        profile = dataset.profile

        # And then change the band count to 1, set the
        # dtype to uint8, and specify LZW compression.
        profile.update(
            dtype=rasterio.float32,
            count=1, # assuming number of band is 1
            transform=new_transform,
            height=int(dataset.height * upscale_factor),
            width=int(dataset.width * upscale_factor),
            nodata=None,
            compress='lzw')

        with rasterio.open(OUTPUT_FILE, 'w', **profile) as dst:
            dst.write(data)
